In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import mysql.connector

In [2]:
def openConnection():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="1998.Ronc@",
        database="futebol_data"
    )
    return conn

def closeConnection(conn):
    conn.close()

#Função para traduzir sigla de nacionalidade
def translate_country(country):
    country_dic = {
        'NED': 'Holanda',
        'EGY': 'Egito',
        'ENG': 'Inglaterra',
        'FRA': 'França',
        'HUN': 'Hungria',
        'SCO': 'Escocia',
        'ARG': 'Argentina',
        'BRA': 'Brasil',
        'COL': 'Colombia',
        'IRL': 'Irlanda',
        'URU': 'Uruguai',
        'GRE': 'Grecia',
        'ITA': 'Italia',
        'CZE': 'Republica tcheca',
        'JPN': 'Japao',
        'POR': 'Portugal',
        'NIR': 'Irlanda do norte',
        'NOR': 'Noruega',
        'GER': 'Alemanha',
        'CRO': 'Croacia',
        'BEL': 'Belgica'
    }
    return country_dic.get(country, 'Não identificado')


#Função para traduzir sigla de posição
def translate_pos(pos):
    pos_dic = {
        'GK': 'Goleiro',
        'LF': 'Lateral esquerdo',
        'DF': 'Zagueiro',
        'RB': 'Lateral direito',
        'FB': 'Lateral',
        'CB': 'Zagueiro central',
        'DM': 'Volante',
        'CM': 'Meio-campista Central',
        'LM': 'Meio-campista Esquerdo',
        'RM': 'Meio-campista Direito',
        'WM': 'Meio-campista Aberto',
        'MF': 'Meio campo',
        'AM':  'Meio-campista Ofensivo',
        'LW': 'Ponta Esquerdo',
        'RW': 'Ponta Direito',
        'FW': 'Atacante',
    }
    return pos_dic.get(pos, 'Posição inválida')


#Função para tratar nome do time
def fetch_team_name(team_name):
    team_dic = {
        'Liverpool': 'Liverpool',
        'Manchester City': 'Manchester City',
        'Arsenal': 'Arsenal',
        'Aston Villa': 'Aston Villa',
        'Brighton and Hove Albion': 'Brighton',
        'Brighton': 'Brighton',
        'Chelsea': 'Chelsea',
        'Tottenham Hotspur': 'Tottenham',
        'Tottenham': 'Tottenham',
        'Newcastle United': 'Newcastle Utd',
        'Newcastle Utd': 'Newcastle Utd',
        'Fulham': 'Fulham',
        'Bournemouth': 'Bournemouth',
        'Manchester United': 'Manchester Utd',
        'Manchester Utd': 'Manchester Utd',
        "Nottingham Forest": "Nott'ham Forest",
        "Nott'ham Forest": "Nott'ham Forest",
        'Brentford': 'Brentford',
        'Leicester City': 'Leicester City',
        'West Ham United': 'West Ham',
        'West Ham': 'West Ham',
        'Everton': 'Everton',
        'Ipswich Town': 'Ipswich Town',
        'Crystal Palace': 'Crystal Palace',
        'Southampton': 'Southampton',
        'Wolverhampton Wanderers': 'Wolves',
        'Wolves': 'Wolves'
    }
    return team_dic.get(team_name, 'Time inválido')

# Função tradução dos dias da semana
def translate_day(day_code):
    week_days = {
        'Mon': 'Segunda-feira',
        'Tue': 'Terça-feira',
        'Wed': 'Quarta-feira',
        'Thu': 'Quinta-feira',
        'Fri': 'Sexta-feira',
        'Sat': 'Sábado',
        'Sun': 'Domingo'
    } 
    return week_days.get(day_code, 'Sigla inválida')


#Função de tradução do resultado da partida
def translate_result(result):
    results = {
        'W': 'Vitória',
        'D': 'Empate',
        'L': 'Derrota'
    }
    return results.get(result, 'Sigla inválida')


# Função para garantir a conversão para inteiro, tratando erros
def safe_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return 0 

In [3]:

standings_url = "https://fbref.com/en/comps/9/Premier-League-Stats"

In [4]:
data = requests.get(standings_url)

In [5]:
soup = BeautifulSoup(data.text)
standings_table = soup.select('table.stats_table')[0]
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if '/squads/' in l]
rows = standings_table.find_all('tr')

In [7]:
conn = openConnection()
cursor = conn.cursor()

for i, row in enumerate(rows[1:], start=1):  # Ignora o cabeçalho
    columns = row.find_all('td')
    column_data = [col.text.strip() for col in columns]
    team_name = column_data[0]  # Nome do time
    
    # Dados para inserção/atualização
    data_to_upsert = (
        team_name,            # Nome do time
        int(column_data[1]),  # Jogos disputados
        int(column_data[2]),  # Vitórias
        int(column_data[3]),  # Derrotas
        int(column_data[4]),  # Empates
        int(column_data[5]),  # Gols marcados
        int(column_data[6]),  # Gols sofridos
        int(column_data[7]),  # Saldo de gols
        int(column_data[8]),  # Pontos
        i                     # Posição do time
    )
    
    # Verifica se o time já existe no banco
    sql_check = "SELECT COUNT(*) FROM team WHERE team_name = %s"
    cursor.execute(sql_check, (team_name,))
    team_exists = cursor.fetchone()[0] > 0

    if team_exists:
        # Atualiza os dados do time existente
        sql_update = """
        UPDATE team
        SET matches_played = %s, win = %s, loss = %s, draw = %s, goals_for = %s, goals_against = %s,
            goals_difference = %s, points = %s, position = %s
        WHERE team_name = %s
        """
        cursor.execute(sql_update, data_to_upsert[1:] + (team_name,))
    else:
        # Insere um novo time
        sql_insert = """
        INSERT INTO team (team_name, matches_played, win, loss, draw, goals_for, goals_against, 
                          goals_difference, points, position)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(sql_insert, data_to_upsert)

# Commit das alterações
conn.commit()
# Fechando a conexão
cursor.close()
closeConnection(conn)

In [ ]:
#tornando os links em formato completo
team_urls = [f"https://fbref.com{l}" for l in links]
team_urls

In [ ]:
# Extração e atualização das informações dos jogadores dos times
conn = openConnection()
cursor = conn.cursor(buffered=True)

for team_url in team_urls:
    data = requests.get(team_url)
    players = pd.read_html(data.text, match="Standard Stats")[0]
    players.columns = players.columns.droplevel(0)
    players.columns = players.columns.get_level_values(-1)
    players = players[:-2]
    players.columns = [f"{col}_{i}" if players.columns.duplicated()[i] else col for i, col in enumerate(players.columns)]
    team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
    sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
    cursor.execute(sql_team_id, (team_name,))
    team_id = cursor.fetchone()

    for index, row in players.iterrows():
        player_name = row['Player']
        player_values = (
            team_id[0],
            translate_country(row['Nation'].split()[-1]),  # Traduzir nacionalidade
            translate_pos(row['Pos'].split(',')[0].strip()),
            safe_int(row['Age'].split('-')[0]),  # Idade
            safe_int(row['MP']),  # Jogos disputados
            safe_int(row['Starts']),  # Começou jogos
            safe_int(row['Min']),  # Minutos jogados
            safe_int(row['Gls']),  # Gols
            safe_int(row['Ast']),  # Assistências
            safe_int(row['PK']) if 'PK' in row else 0,  # Penaltys convertidos em gol, 0 se não existir
            safe_int(row['PKatt']) if 'PKatt' in row else 0,  # Penaltys tentados, 0 se não existir
            safe_int(row['CrdY']) if 'CrdY' in row else 0,  # Cartões amarelos, 0 se não existir
            safe_int(row['CrdR']) if 'CrdR' in row else 0   # Cartões vermelhos, 0 se não existir
        )
        
        # Verifica se o jogador já existe
        sql_check_player = "SELECT id FROM Player WHERE player_name = %s AND Team_id = %s"
        cursor.execute(sql_check_player, (player_name, team_id[0]))
        existing_player = cursor.fetchone()

        if existing_player:
            # Atualiza as informações do jogador se ele já existir
            sql_update = """
                UPDATE Player 
                SET 
                    country = %s, 
                    position = %s, 
                    age = %s, 
                    matches_played = %s, 
                    started = %s, 
                    minutes = %s, 
                    goals = %s, 
                    assist = %s, 
                    penaltys_made = %s, 
                    penaltys_attempted = %s, 
                    yellow_card = %s, 
                    red_card = %s 
                WHERE id = %s
            """
            cursor.execute(sql_update, (*player_values[1:], existing_player[0]))
        else:
            # Insere um novo jogador se ele não existir
            sql_insert = """
                INSERT INTO Player (player_name, Team_id, country, position, age, matches_played, started, minutes, goals, assist, penaltys_made, penaltys_attempted, yellow_card, red_card)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql_insert, (player_name, *player_values))

    time.sleep(5)  # Evita sobrecarregar o site com requisições consecutivas

conn.commit()
cursor.close()
closeConnection(conn)

In [ ]:
# Pegando informações das partidas dos times na Premier League e atualizando os valores
conn = openConnection()
cursor = conn.cursor(buffered=True)
matches_array = []

for team_url in team_urls:
    print(team_url)
    data = requests.get(team_url)
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    premier_league_matches = matches[matches['Comp'] == 'Premier League']
    
    for index, row in premier_league_matches.iterrows():
        venue = 'Casa' if row['Venue'] == 'Home' else 'Fora'
        opponent_name = fetch_team_name(row['Opponent'])
        captain_name = row['Captain']

        # Busca o ID do oponente
        sql_opponent_id = "SELECT id FROM team WHERE team_name = %s"
        cursor.execute(sql_opponent_id, (opponent_name,))
        opponent_result = cursor.fetchone()
        opponent_id = opponent_result[0] if opponent_result else None

        # Busca o ID do capitão
        captain_id = None
        if isinstance(captain_name, str):
            sql_captain_id = "SELECT id FROM player WHERE player_name = %s"
            cursor.execute(sql_captain_id, (captain_name,))
            captain_result = cursor.fetchone()
            captain_id = captain_result[0] if captain_result else None

        week_day = translate_day(row['Day'].strip())
        result = translate_result(row['Result'].strip()) if isinstance(row["Result"], str) else None
        goals_for = row['GF'] if not pd.isna(row['GF']) else None
        goals_against = row['GA'] if not pd.isna(row['GA']) else None
        poss = row['Poss'] if not pd.isna(row['Poss']) else None
        formation = row['Formation'] if isinstance(row['Formation'], str) else None
        opp_formation = row['Opp Formation'] if isinstance(row['Opp Formation'], str) else None
        
        team_name = fetch_team_name(team_url.split('/')[-1].replace('-Stats', '').replace('-', ' '))
        
        # Busca o ID do time
        sql_team_id = "SELECT id FROM Team WHERE team_name = %s"
        cursor.execute(sql_team_id, (team_name,))
        team_result = cursor.fetchone()
        team_id = team_result[0] if team_result else None
        
        # Verifica se a partida já existe no banco de dados
        sql_check = "SELECT COUNT(*) FROM Matches WHERE date = %s AND opponent_team_id = %s"
        cursor.execute(sql_check, (row['Date'], opponent_id))
        match_exists = cursor.fetchone()[0] > 0

        if match_exists:
            # Atualiza a partida existente
            match_info = (
            row['Date'], row['Round'], week_day, venue, result, goals_for, goals_against, 
             poss, captain_id, formation, opp_formation, team_id
        )
            sql_update = """
            UPDATE Matches 
            SET 
                round = %s, 
                week_day = %s, 
                venue = %s, 
                result = %s, 
                goals_for = %s, 
                goals_against = %s, 
                poss = %s, 
                captain_id = %s, 
                formation = %s, 
                opponent_formation = %s, 
                team_id = %s
            WHERE date = %s AND opponent_team_id = %s
            """
            update_params = match_info[1:] + (row['Date'], opponent_id)
            cursor.execute(sql_update, update_params)
        else:
            # Insere nova partida
            match_info = (
            row['Date'], row['Round'], week_day, venue, result, goals_for, goals_against, 
            opponent_id, poss, captain_id, formation, opp_formation, team_id
        )
            sql_insert = """
            INSERT INTO Matches (date, round, week_day, venue, result, goals_for, goals_against, opponent_team_id, 
                                 poss, captain_id, formation, opponent_formation, team_id)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            """
            cursor.execute(sql_insert, match_info)

    time.sleep(5)

conn.commit()
cursor.close()
closeConnection(conn)
